In [1]:
import jutil
import pandas as pd

In [2]:
sql = "select DISTINCT(course_name),term_order from view_stu_course_mark where course_type='必'  and speciality_code='0101'"
course = jutil.load_data(sql)

In [4]:
cdf = pd.DataFrame(course)
cdf.head()

,course_name,term_order
0,体育Ⅲ,-2
1,建筑设计基础AI,-1
2,军事理论,-1
3,思想道德修养B,-1
4,形势与政策1,-1


In [2]:
sql2 = "select * from view_stu_course_mark where course_type='必' and speciality_code='0101'"
data = jutil.load_pd_df(sql2)

In [4]:
data.head()

,college_code,course_code,course_name,course_type,mark,pmark,speciality_code,speciality_name,student_id,term_order
0,01,6111120,大学物理实验Ⅱ,必,5,73.0,0101,土木工程,19950854,-2
1,01,9010110,体育Ⅰ,必,7,80.0,0101,土木工程,19950854,-3
2,01,6111120,大学物理实验Ⅱ,必,1,61.0,0101,土木工程,19960006,2
3,01,9010120,体育Ⅱ,必,6,76.0,0101,土木工程,19960006,2
4,01,7035000,马克思主义哲学原理,必,8,83.0,0101,土木工程,19960037,4


In [27]:
# 把数据写入文件,方便下次调用
data.to_csv(path_or_buf="000-mark-0101.csv")

In [6]:
dg = data.groupby('student_id').groups
type(dg)

dict

In [18]:
input_list = []
for k in dg:
    ids = dg[k]
    items = [ data.iloc[id].get('course_code')+'_'+str(data.iloc[id].get('mark')) for id in ids]
    items = list(set(items)) # 去重
    input_list.append((k,items))

In [19]:
len(input_list)

11357

In [20]:
input_list[0]

('19950854',
 ['0384448_1',
  '6111120_5',
  '9010120_5',
  '8010220_4',
  '9010140_7',
  '3046108_1',
  '9010130_7',
  '9010110_7',
  '8010210_9',
  '8010230_4',
  '6111110_9',
  '8010240_6'])

In [15]:
from pyspark.ml.fpm import FPGrowth

In [21]:
ddd = spark.createDataFrame(input_list,['id','items'])

fpG = FPGrowth(itemsCol="items",minSupport=0.3,minConfidence=0.7)
model = fpG.fit(ddd)

In [29]:
model.freqItemsets.show(n=10,truncate=False)

+---------------------------------+----+
|items                            |freq|
+---------------------------------+----+
|[7001155_9]                      |4640|
|[7001155_9, 7001156_9]           |3928|
|[7001155_9, 7001156_9, 7001154_9]|3647|
|[7001155_9, 7001154_9]           |4189|
|[7001155_9, 7001154_9, 7001152_9]|3579|
|[7001155_9, 7001153_9]           |3676|
|[7001155_9, 7001153_9, 7001152_9]|3662|
|[7001155_9, 7001152_9]           |4026|
|[7001152_9]                      |5634|
|[7001150_9]                      |4481|
+---------------------------------+----+
only showing top 10 rows



In [30]:
model.associationRules.show(truncate=False)

+----------------------+-----------+------------------+
|antecedent            |consequent |confidence        |
+----------------------+-----------+------------------+
|[7001156_9, 7001154_9]|[7001155_9]|0.9460440985732814|
|[7001153_9, 7001152_9]|[7001155_9]|0.7924691625189353|
|[7001153_9, 7001152_9]|[7001154_9]|0.8255788790305129|
|[7001153_9]           |[7001155_9]|0.77997029492892  |
|[7001153_9]           |[7001150_9]|0.7307447485677913|
|[7001153_9]           |[7001154_9]|0.8126458731169107|
|[7001153_9]           |[7001152_9]|0.9804795247188627|
|[7001150_9, 7001152_9]|[7001154_9]|0.820952380952381 |
|[7001150_9, 7001152_9]|[7001151_9]|0.8571428571428571|
|[7001155_9, 7001152_9]|[7001154_9]|0.8889716840536512|
|[7001155_9, 7001152_9]|[7001153_9]|0.9095876800794833|
|[7001153_9, 7001154_9]|[7001152_9]|0.9960835509138382|
|[7001150_9, 7001154_9]|[7001152_9]|0.9945197577156043|
|[7001154_9, 7001152_9]|[7001155_9]|0.8080830887333483|
|[7001154_9, 7001152_9]|[7001150_9]|0.7785053059

In [25]:
pdc = model.associationRules.toPandas()

In [28]:
pdc.head(50)

,antecedent,consequent,confidence
0,"[7001156_9, 7001154_9]",[7001155_9],0.946044
1,"[7001153_9, 7001152_9]",[7001155_9],0.792469
2,"[7001153_9, 7001152_9]",[7001154_9],0.825579
3,[7001153_9],[7001155_9],0.779970
4,[7001153_9],[7001150_9],0.730745
5,[7001153_9],[7001154_9],0.812646
6,[7001153_9],[7001152_9],0.980480
7,"[7001150_9, 7001152_9]",[7001154_9],0.820952
8,"[7001150_9, 7001152_9]",[7001151_9],0.857143
9,"[7001155_9, 7001152_9]",[7001154_9],0.888972


In [ ]:
# 根据课程先修关系去除一些置信度

